<a href="https://colab.research.google.com/github/hiiragi-kaede/chainer_practice/blob/master/practice.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import cupy as cp
import time
import tabulate

def get_w_np(x,t):
  xx=np.dot(x.T,x)
  xx_inv=np.linalg.inv(xx)
  xt=np.dot(x.T,t)
  w=np.dot(xx_inv,xt)
  return w

def get_w_cp(x,t):
  xx=cp.dot(x.T,x)
  xx_inv=cp.linalg.inv(xx)
  xt=cp.dot(x.T,t)
  w=cp.dot(xx_inv,xt)
  return w

times_cpu=[]
for N in [10,100,1000,10000]:
  np.random.seed(0)
  x=np.random.rand(N,N)
  t=np.random.rand(N,1)

  t_start=time.time()

  w=get_w_np(x,t)

  t_end=time.time()
  elapsed_time=t_end-t_start
  print('N={:>5}:{:>8.5f} sec'.format(N,elapsed_time))
  times_cpu.append(elapsed_time)

times_gpu=[]
for N in [10,100,1000,10000]:
  cp.random.seed(0)
  x=cp.random.rand(N,N)
  t=cp.random.rand(N,1)

  cp.cuda.Stream.null.synchronize()
  t_start=time.time()

  w=get_w_cp(x,t)

  cp.cuda.Stream.null.synchronize()
  t_end=time.time()
  elapsed_time=t_end-t_start
  print('N={:>5}:{:>8.5f} sec'.format(N,elapsed_time))
  times_gpu.append(elapsed_time)

N=[10,100,1000,10000]
times_cpu=np.asarray(times_cpu)
times_gpu=np.asarray(times_gpu)
ratio=['{:.2f} x'.format(r) for r in times_cpu/times_gpu]

table=tabulate.tabulate(
    zip(N,times_cpu,times_gpu,ratio),
    headers=['N','Numpyでの実行時間(sec)','Cupyでの実行時間(sec)','高速化倍率']
)
print(table)

N=   10: 0.00033 sec
N=  100: 0.00075 sec
N= 1000: 0.12585 sec
N=10000:90.99724 sec
N=   10: 0.00056 sec
N=  100: 0.00090 sec
N= 1000: 0.01531 sec
N=10000: 1.34605 sec
    N    Numpyでの実行時間(sec)    Cupyでの実行時間(sec)  高速化倍率
-----  ------------------------  -----------------------  ------------
   10               0.000325918              0.000562906  0.58 x
  100               0.000751019              0.000895023  0.84 x
 1000               0.125846                 0.0153127    8.22 x
10000              90.9972                   1.34605      67.60 x


In [0]:
from sklearn.datasets import load_boston
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PowerTransformer
from sklearn.pipeline import Pipeline

dataset=load_boston()
x=dataset.data
t=dataset.target
pipeline=Pipeline([
  ('scaler',PowerTransformer()),
  ('reg',LinearRegression())
])
x_train,x_test,t_train,t_test=train_test_split(x,t,test_size=0.3,random_state=0)

pipeline.fit(x_train,t_train)
pipeline.score(x_train,t_train)
linear_result=pipeline.score(x_test,t_test)
print(linear_result)

0.7002856551689581


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_data.py:2982: RuntimeWarning: divide by zero encountered in log
  loglike = -n_samples / 2 * np.log(x_trans.var())


In [0]:
num=[x for x in range(2,101)]
isprime=[True for x in range(2,101)]
for i in range(2,11):
  id=i
  while id<=100-i:
    id+=i
    isprime[id-2]=False
  
res=[x for x in num if isprime[x-2]]
print(res)

[2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]


In [0]:
def f(a):
  a=[6,7,8]
  print(id(a))

def g(a):
  a.append(1)

def func():
  a0=[1,2,3]
  print(id(a0))
  f(a0)
  
  print(a0)

  a1=[1,2,3]
  g(a1)
  print(a1)

func()

139656822769288
139656822786632
[1, 2, 3]
[1, 2, 3, 1]


In [0]:
class DataManager:
  def __init__(self,x,y,z):
    self.x=x
    self.y=y
    self.z=z
  
  def add_x(self,delta):
    self.x+=delta
  
  def add_y(self,delta):
    self.y+=delta
  
  def add_z(self,delta):
    self.z+=delta
  
  def sum(self):
    return self.x+self.y+self.z

data=DataManager(2,3,5)
print(data.sum())
data.add_x(4)
print(data.sum())
data.add_y(0)
print(data.sum())
data.add_z(-9)
print(data.sum())
print('{},{},{}'.format(data.x,data.y,data.z))

10
14
14
5
6,3,-4
